In [2]:
#IMPORTS AND FILE GENERATION
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.draw import disk, circle_perimeter
from scipy.ndimage import gaussian_filter
import os
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance, ImageChops
import matplotlib.pyplot as plt
import random


for folder_path in ['masks', 'genimages']:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")


Folder 'masks' created successfully.
Folder 'genimages' created successfully.


In [9]:
#SYNTHETIC FLUORO IMAGE GENERATION
def circles_overlap(circle1, circle2):
    x1, y1, r1 = circle1
    x2, y2, r2 = circle2
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance < (r1 + r2) * 0.8  # Adjust the overlap threshold as needed

# Function to generate a more realistic GUV image with variable background shades within the image
def generate_F_image(params):
    # Create a background with a gradient of grey shades
    x = np.linspace(0, 1, params['image_size'][1])
    y = np.linspace(0, 1, params['image_size'][0])
    X, Y = np.meshgrid(x, y)
    random_matrix = np.random.uniform(0, 1, size=params['image_size'])
    smooth_random_matrix = gaussian_filter(random_matrix, sigma=10)
    background_gradient = smooth_random_matrix
    num = np.random.uniform(0, 0.26)
    num1 = np.random.uniform(0.03, 0.15)
    background = np.random.uniform(num, (num+num1), params['image_size']) + background_gradient
    background_black = np.zeros(params['image_size'], dtype=float)
    image_mask = np.clip(background_black, 0, 1)
    image = np.clip(background, 0, 1)  # Ensure the background is within the valid range

    # Store the circle data
    circle_data = []
    i=0

    # Generate GUVs
    circle_data = generate_guvs(params, image, circle_data, i)

    # Generate small GUVs
    circle_data = generate_small_guvs(params, image, circle_data, i)

    return image, circle_data

def generate_guvs(params, image, circle_data, i):
    num_guvs = np.random.randint(params['min_num_guvs'], params['max_num_guvs'])
    while len(circle_data) < num_guvs:
        # Randomly select the radius and intensity of the GUV
        background_black = np.zeros(params['image_size'], dtype=float)
        image_mask = np.clip(background_black, 0, 1)
        radius = np.random.randint(params['guv_min_size'], params['guv_max_size'])
        intensity = np.random.uniform(0.18, 1)  # Brighter intensity for GUVs

        # Randomly select the position ensuring it is within the image bounds
        x_center = np.random.randint(radius, params['image_size'][1])
        y_center = np.random.randint(radius, params['image_size'][0])

        # Use skimage to generate circle coordinates
        rr, cc = disk((y_center, x_center), radius, shape=params['image_size'])

        # Check if the new circle overlaps with any existing circles
        overlap = np.any([circles_overlap(existing_circle, (x_center, y_center, radius)) for existing_circle in circle_data])

        # Add the circle to the image if there is no overlap
        if not overlap:
            # Add some gradient to the GUV intensity to simulate uneven illumination
            distance_to_center = np.sqrt((rr - y_center)**2 + (cc - x_center)**2)
            gradient = (1 - (distance_to_center / radius)) ** 0.5  # Quadratic gradient
            image[rr, cc] = np.clip(image[rr, cc] + intensity * gradient, 0, 1)
            image_mask[rr, cc] = 1
            cv2.imwrite('masks/'+str(line)+'_'+str(i)+'.png', image_mask*255)

            # Store the circle data
            circle_data.append((x_center, y_center, radius))
            i+=1
    return circle_data

def generate_small_guvs(params, image, circle_data, i):

    num_small_guvs = np.random.randint(params['min_num_small_guvs'], params['max_num_small_guvs'])
    while len(circle_data) < num_small_guvs:
        background_black = np.zeros(params['image_size'], dtype=float)
        image_mask = np.clip(background_black, 0, 1)
        # Randomly select the radius
        radius = np.random.randint(params['guv_small_min_size'], params['guv_small_max_size'])

        # Randomly select the position ensuring it is within the image bounds
        x_center = np.random.randint(radius, params['image_size'][1] - radius)
        y_center = np.random.randint(radius, params['image_size'][0] - radius)

        # Use skimage to generate circle coordinates
        rr, cc = disk((y_center, x_center), radius, shape=params['image_size'])

        # Check if the new circle overlaps with any existing circles
        overlap = np.any([np.sqrt((x_center - existing_circle[0])**2 + (y_center - existing_circle[1])**2) < radius + existing_circle[2] for existing_circle in circle_data])

        if not overlap:
            # Calculate the distance to the center for each point
            distance_to_center = np.sqrt((rr - y_center)**2 + (cc - x_center)**2)

            # Define masks for the perimeter and interior
            perimeter_mask = np.logical_and(distance_to_center >= radius - params['perimeter_width'], distance_to_center <= radius)
            interior_mask = distance_to_center < radius - params['perimeter_width']

            # Darkening factors
            interior_darkening_factor = np.random.uniform(1.08, 1.15)
            perimeter_darkening_factor = interior_darkening_factor  # For the perimeter, slightly less dark than the interior

            # Darken the interior of the circle slightly
            image[rr[interior_mask], cc[interior_mask]] *= interior_darkening_factor

            # Darken the perimeter slightly
            image[rr[perimeter_mask], cc[perimeter_mask]] *= perimeter_darkening_factor
            image_mask[rr[perimeter_mask], cc[perimeter_mask]]
            image_mask[rr[interior_mask], cc[interior_mask]]= 1
            cv2.imwrite('masks/'+str(line)+'_'+str(i)+'.png', image_mask*255)

            # Store the circle data
            circle_data.append((x_center, y_center, radius))
            i+=1
    return circle_data

# Parameters for image generation
params = {
    'image_size': (1024, 1280),  # Size of the image, width x height
    'min_num_guvs': 10,  # Minimum number of GUVs to generate in each image
    'max_num_guvs': 100,  # Approximate number of GUVs to generate in each image
    'guv_min_size': 10,  # Minimum radius of GUVs
    'guv_max_size': 35,  # Maximum radius of GUVs
    'min_num_small_guvs': 10,
    'max_num_small_guvs': 400,
    'guv_small_min_size': 8,
    'guv_small_max_size': 12,
    'perimeter_width': 2,
    'num_images': 1  # Number of images to generate
}



for line in range(params['num_images']):
    # Generate a single image with variable background shade and display it
    img, circle_data = generate_F_image(params)
    cv2.imwrite('genimages/'+str(line)+'.png', img*255)
    print('Generated image ' + str(line) + ' with ' + str(len(circle_data)) + ' GUVs.')

Generated image 0 with 393 GUVs.


In [13]:
#SYNTHETIC BRIGHTFIELD IMAGE GENERATION

def circles_overlap(circle1, circle2):
    x1, y1, r1 = circle1
    x2, y2, r2 = circle2
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance < (r1 + r2) * 0.65  # Adjust the overlap threshold as needed

def apply_gaussian_blur(image, mask, sigma=2):
    """Apply Gaussian blur to specific regions of an image defined by a mask."""
    blurred_image = gaussian_filter(image, sigma=sigma)
    return np.where(mask, blurred_image, image)

def generate_BF_image(params):
    image_size = params['image_size']
    num_guvs = params['num_guvs']
    guv_min_size = params['guv_min_size']
    guv_max_size = params['guv_max_size']
    perimeter_width = params['perimeter_width']
    num_oil_droplets = params['num_oil_droplets']
    oil_droplet_min_size = params['oil_droplet_min_size']
    oil_droplet_max_size = params['oil_droplet_max_size']
    oil_perimeter_width = params['oil_perimeter_width']
    oil_perimeter_blur_sigma = params['oil_perimeter_blur_sigma']
    num_lipid_droplets = params['num_lipid_droplets']
    lipid_droplet_min_size = params['lipid_droplet_min_size']
    lipid_droplet_max_size = params['lipid_droplet_max_size']
    lipid_perimeter_width = params['lipid_perimeter_width']

    # Create a background with a gradient of grey shades
    x = np.linspace(0, 1, image_size[1])
    y = np.linspace(0, 1, image_size[0])
    X, Y = np.meshgrid(x, y)
    random_matrix = np.random.uniform(0, 1, size=image_size)
    smooth_random_matrix = gaussian_filter(random_matrix, sigma=10)
    background_gradient = smooth_random_matrix
    num = np.random.uniform(0, 0.26)
    num1 = np.random.uniform(0.03, 0.15)
    background = np.random.uniform(num, (num+num1), image_size) + background_gradient
    
    image = np.clip(background, 0, 1)  # Ensure the background is within the valid range
    
    # Store the circle data
    circle_data = []
    i=0
    while len(circle_data) < num_guvs:
        background_black = np.zeros(image_size, dtype=float)
        image_mask = np.clip(background_black, 0, 1)
        # Randomly select the radius
        radius = np.random.randint(guv_min_size, guv_max_size)

        # Randomly select the position ensuring it is within the image bounds
        x_center = np.random.randint(radius, image_size[1] - radius)
        y_center = np.random.randint(radius, image_size[0] - radius)

        # Use skimage to generate circle coordinates
        rr, cc = disk((y_center, x_center), radius, shape=image_size)

        # Check if the new circle overlaps with any existing circles
        overlap = np.any([np.sqrt((x_center - existing_circle[0])**2 + (y_center - existing_circle[1])**2) < radius + existing_circle[2] for existing_circle in circle_data])

        if not overlap:
            # Calculate the distance to the center for each point
            distance_to_center = np.sqrt((rr - y_center)**2 + (cc - x_center)**2)

            # Define masks for the perimeter and interior
            perimeter_mask = np.logical_and(distance_to_center >= radius - perimeter_width, distance_to_center <= radius)
            interior_mask = distance_to_center < radius - perimeter_width

            # Darkening factors
            interior_darkening_factor = 0.93  # For the interior
            perimeter_darkening_factor = 0.90  # For the perimeter, slightly less dark than the interior

            # Darken the interior of the circle slightly
            image[rr[interior_mask], cc[interior_mask]] *= interior_darkening_factor

            # Darken the perimeter slightly
            image[rr[perimeter_mask], cc[perimeter_mask]] *= perimeter_darkening_factor
            image_mask[rr[perimeter_mask], cc[perimeter_mask]]
            image_mask[rr[interior_mask], cc[interior_mask]]= 1
            cv2.imwrite('masks/'+str(line)+'_'+str(i)+'g.png', image_mask*255)
            # Store the circle data
            circle_data.append((x_center, y_center, radius))
            i+=1
    i=0
    for _ in range(num_oil_droplets):
            background_black = np.zeros(image_size, dtype=float)
            image_mask = np.clip(background_black, 0, 1)
            # Randomly select the radius
            radius = np.random.randint(oil_droplet_min_size, oil_droplet_max_size)

            # Randomly select the position
            x_center = np.random.randint(0, image_size[1])
            y_center = np.random.randint(0, image_size[0])

            # Generate oil droplet
            rr, cc = disk((y_center, x_center), radius, shape=image_size)
            image[rr, cc] = 1  # White interior

            # Generate wider perimeter for blurring
            rr_perim, cc_perim = disk((y_center, x_center), radius + oil_perimeter_width, shape=image_size)
            perimeter_mask = np.zeros_like(image, dtype=bool)
            perimeter_mask[rr_perim, cc_perim] = True  # Mark the perimeter region
            
            # Exclude the interior from the perimeter mask to blur only the perimeter
            rr_interior, cc_interior = disk((y_center, x_center), radius, shape=image_size)
            perimeter_mask[rr_interior, cc_interior] = False

            # Apply Gaussian blur to the perimeter
            image = apply_gaussian_blur(image, perimeter_mask, sigma=oil_perimeter_blur_sigma)
            
            # Darken the blurred perimeter slightly
            darkening_factor = 0.9 # Darken perimeter
            image = np.where(perimeter_mask, image * darkening_factor, image)
            
            image_mask[rr, cc] = 1
            image_mask = np.where(perimeter_mask, 1, image_mask)
            cv2.imwrite('masks/'+str(line)+'_'+str(i)+'o.png', image_mask*255)
            i+=1
    group_size_range = (lipid_droplet_min_size, lipid_droplet_max_size)  # Range for number of droplets in each group
    group_centers = []  # List to hold the center and reach of each group
    i=0
    for _ in range(num_lipid_droplets):
        background_black = np.zeros(image_size, dtype=float)
        image_mask = np.clip(background_black, 0, 1)
        placed = False
        while not placed:
            # Determine the number of droplets in this group
            num_droplets_in_group = np.random.randint(*group_size_range)
            
            # Tentatively select a central position for the group
            group_x_center = np.random.randint(0, image_size[1])
            group_y_center = np.random.randint(0, image_size[0])
            max_radius = lipid_droplet_max_size + lipid_perimeter_width
            offset_range = max_radius  # Maximum distance a droplet can be from the group center

            # Check if the new group intersects with existing groups
            intersects = False
            for center, reach in group_centers:
                distance = np.sqrt((group_x_center - center[0])**2 + (group_y_center - center[1])**2)
                if distance < (reach + offset_range):
                    intersects = True
                    break

            if not intersects:
                placed = True
                # Record the center and reach of the new group
                group_centers.append(((group_x_center, group_y_center), offset_range + max_radius))

                for _ in range(num_droplets_in_group):
                    # Adjustments for droplet placement within the group, similar to original code
                    x_offset = np.random.randint(-offset_range, offset_range)
                    y_offset = np.random.randint(-offset_range, offset_range)
                    x_center = max(0, min(image_size[1] - 1, group_x_center + x_offset))
                    y_center = max(0, min(image_size[0] - 1, group_y_center + y_offset))
                    interior_darkening_factor = np.random.uniform(0.4, 0.55)

                    # Droplet generation and perimeter handling, similar to original code
                    rr, cc = disk((y_center, x_center), radius, shape=image_size)
                    image[rr, cc] *= interior_darkening_factor  # Adjust droplet interior
                    # Further processing for perimeter blur and darkening, similar to original code
                    rr_perim, cc_perim = disk((y_center, x_center), radius + lipid_perimeter_width, shape=image_size)
                    perimeter_mask = np.zeros_like(image, dtype=bool)
                    perimeter_mask[rr_perim, cc_perim] = True
                    
                    # Exclude the interior from the perimeter mask to blur only the perimeter
                    rr_interior, cc_interior = disk((y_center, x_center), radius, shape=image_size)
                    perimeter_mask[rr_interior, cc_interior] = False

                    # Apply Gaussian blur to the perimeter
                    image = apply_gaussian_blur(image, perimeter_mask, sigma=1.2)
                    
                    # Darken the blurred perimeter slightly
                    darkening_factor = np.random.uniform(0.6, 0.7)  # Darken perimeter
                    image = np.where(perimeter_mask, image * darkening_factor, image)
                    image_mask[rr, cc] = 1
                    image_mask = np.where(perimeter_mask, 1, image_mask)
                cv2.imwrite('masks/'+str(line)+'_'+str(i)+'l.png', image_mask*255)
                i+=1
    return image

num_images = 1
for line in range(num_images):
    params = {
        'image_size': (1024, 1280),
        'num_guvs': np.random.randint(100, 500),
        'guv_min_size': 7,
        'guv_max_size': 45,
        'perimeter_width': 3,
        'num_oil_droplets': np.random.randint(20, 60),
        'oil_droplet_min_size': 2,
        'oil_droplet_max_size': 4,
        'oil_perimeter_width': 7,
        'oil_perimeter_blur_sigma': 0.1,
        'num_lipid_droplets': np.random.randint(10, 30),
        'lipid_droplet_min_size': 8,
        'lipid_droplet_max_size': 11,
        'lipid_perimeter_width': 5}
    img = generate_BF_image(params)
    cv2.imwrite('genimages/'+str(line)+'.png', img*255)
    print('Generated image ' + str(line))

Generated image 0


In [21]:
#SEMISYNTHETIC BF GENERATION
def circles_overlap(circle1, circle2):
    x1, y1, r1 = circle1
    x2, y2, r2 = circle2
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance < (r1 + r2) * 0.6 # Adjust the overlap threshold as needed

def apply_gaussian_blur(image, mask, sigma=2):
    """Apply Gaussian blur to specific regions of an image defined by a mask."""
    blurred_image = gaussian_filter(image, sigma=sigma)
    return np.where(mask, blurred_image, image)

def gen_BF_SS_image(params):
    image_size = params['image_size']
    num_guvs = params['num_guvs']
    num_big_guvs = params['num_big_guvs']
    num_LLA = params['num_LLA']
    num_SLA = params['num_SLA']
    num_OD = params['num_OD']
    
    x = np.linspace(0, 1, image_size[1])
    y = np.linspace(0, 1, image_size[0])
    X, Y = np.meshgrid(x, y)
    random_matrix = np.random.uniform(0, 1, size=image_size)
    smooth_random_matrix = gaussian_filter(random_matrix, sigma=10)
    background_gradient = smooth_random_matrix
    num = np.random.uniform(-0.075, -0.125)
    num1 = np.random.uniform(0.08, 0.11)
    background = np.random.uniform(num, (num+num1), image_size) + background_gradient
    image = np.clip(background, 0, 1)  # Ensure the background is within the valid range
    image = Image.fromarray((image * 255).astype(np.uint8))
    
    i = 0
    guv_positions = []  # List to store the positions of generated GUVs
    for _ in range(num_big_guvs):
        black_image = Image.new('L', (1280, 1024), color='black')
        GUV = 0
        img = Image.open(f'GUV/G{GUV}.png').convert("RGBA")
        img = img.rotate(np.random.randint(0, 360), expand=True)
        
        # Generate a random position for the GUV
        position = (np.random.randint(0, image_size[1]-img.width), np.random.randint(0, image_size[0]-img.height))
        
        # Check if the generated position overlaps with any existing GUVs
        overlap = False
        for guv_position in guv_positions:
            if circles_overlap((position[0], position[1], img.width/2), (guv_position[0], guv_position[1], guv_position[2]/2)):
                overlap = True
                break
        
        # If there is an overlap, generate a new position until no overlap is found
        while overlap:
            position = (np.random.randint(0, image_size[1]-img.width), np.random.randint(0, image_size[0]-img.height))
            overlap = False
            for guv_position in guv_positions:
                if circles_overlap((position[0], position[1], img.width/2), (guv_position[0], guv_position[1], guv_position[2]/2)):
                    overlap = True
                    break
        
        # Add the position and width to the list of GUV positions
        guv_positions.append((*position, img.width))
        image.paste(img, position, mask=img)
        img = Image.eval(img, lambda x: x * 500)
        black_image.paste(img, position, mask=img)
        black_image.save('masks/'+str(line)+'_'+str(i)+'g.png')
        i += 1

    for _ in range(num_guvs):
        black_image = Image.new('L', (1280, 1024), color='black')
        GUV = np.random.randint(1, 45)
        img = Image.open(f'GUV/G{GUV}.png').convert("RGBA")
        img = img.rotate(np.random.randint(0, 360), expand=True)
        
        # Generate a random position for the GUV
        position = (np.random.randint(0, image_size[1]-img.width), np.random.randint(0, image_size[0]-img.height))
        
        # Check if the generated position overlaps with any existing GUVs
        overlap = False
        for guv_position in guv_positions:
            if circles_overlap((position[0], position[1], img.width/2), (guv_position[0], guv_position[1], guv_position[2]/2)):
                overlap = True
                break
        
        # If there is an overlap, generate a new position until no overlap is found
        while overlap:
            position = (np.random.randint(0, image_size[1]-img.width), np.random.randint(0, image_size[0]-img.height))
            overlap = False
            for guv_position in guv_positions:
                if circles_overlap((position[0], position[1], img.width/2), (guv_position[0], guv_position[1], guv_position[2]/2)):
                    overlap = True
                    break
        
        # Add the position and width to the list of GUV positions
        guv_positions.append((*position, img.width))
        image.paste(img, position, mask=img)
        img = Image.eval(img, lambda x: x * 500)
        black_image.paste(img, position, mask=img)
        black_image.save('masks/'+str(line)+'_'+str(i)+'g.png')
        i += 1
    i=0
    for _ in range(num_LLA):
        black_image = Image.new('L', (1280, 1024), color='black')
        LLA = np.random.randint(12, 22)
        img = Image.open(f'LA/LA{LLA}.png').convert("RGBA")
        img = img.rotate(np.random.randint(0, 360), expand=True)
        position = (np.random.randint(0, image_size[1]-img.width), np.random.randint(0, image_size[0]-img.height))
        image.paste(img, position, mask=img)
        img = Image.eval(img, lambda x: x * 500)
        black_image.paste(img, position, mask=img)
        black_image.save('masks/'+str(line)+'_'+str(i)+'l.png')
        i+=1
    i=0
    for _ in range(num_SLA):
        black_image = Image.new('L', (1280, 1024), color='black')
        if np.random.rand() < 0.5:
            SLA = np.random.randint(0, 12)  
        else:
            SLA = np.random.randint(22, 50)
        img = Image.open(f'LA/LA{SLA}.png').convert("RGBA")
        img = img.rotate(np.random.randint(0, 360), expand=True)
        position = (np.random.randint(0, image_size[1]-img.width), np.random.randint(0, image_size[0]-img.height))
        image.paste(img, position, mask=img)
        img = Image.eval(img, lambda x: x * 500)
        black_image.paste(img, position, mask=img)
        black_image.save('masks/'+str(line)+'_'+str(i)+'l.png')
        i+=1
    i=0
    for _ in range(num_OD):
        black_image = Image.new('L', (1280, 1024), color='black')
        OD = np.random.randint(0, 17)
        img = Image.open(f'OD/OD{OD}.png').convert("RGBA")
        img = img.rotate(np.random.randint(0, 360), expand=True)
        position = (np.random.randint(0, image_size[1]-img.width), np.random.randint(0, image_size[0]-img.height))
        image.paste(img, position, mask=img)
        img = Image.eval(img, lambda x: x * 500)
        black_image.paste(img, position, mask=img)
        black_image.save('masks/'+str(line)+'_'+str(i)+'o.png')
        i+=1
    return image

num_images = 1
for line in range(num_images):
    params = {
        'image_size': (1024, 1280),
        'num_guvs': np.random.randint(100, 500),
        'num_big_guvs': np.random.randint(0, 1),
        'num_LLA': np.random.randint(10, 20),
        'num_SLA': np.random.randint(10, 20),
        'num_OD': np.random.randint(10, 20)
    }
    img = gen_BF_SS_image(params)
    img.save('genimages/'+str(line)+'.png')
    print('Generated image ' + str(line))


Generated image 0
